# 4장: 데이터와 지시사항 분리하기

- [수업](#lesson)
- [연습 문제](#exercises)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## 수업

종종 우리는 전체 프롬프트를 작성하는 대신, **Claude에 제출하기 전에 추가 입력 데이터로 나중에 수정할 수 있는 프롬프트 템플릿**을 원합니다. 이는 Claude가 매번 동일한 작업을 수행하지만, Claude가 작업에 사용하는 데이터가 매번 다를 수 있는 경우에 유용할 수 있습니다.

다행히도, **프롬프트의 고정된 골격을 가변적인 사용자 입력과 분리한 다음, 전체 프롬프트를 Claude에 보내기 전에 사용자 입력을 프롬프트에 대체**하는 것으로 이를 쉽게 할 수 있습니다.

아래에서는 대체 가능한 프롬프트 템플릿을 작성하는 방법과 사용자 입력을 대체하는 방법을 단계별로 살펴보겠습니다.

### 예제

첫 번째 예제에서는 Claude에게 동물 소리 생성기 역할을 요청합니다. Claude에 제출된 전체 프롬프트는 입력(이 경우 "Cow")으로 대체된 `PROMPT_TEMPLATE`일 뿐입니다. 전체 프롬프트를 출력할 때 f-문자열을 통해 `ANIMAL` 자리 표시자가 "Cow"라는 단어로 대체되는 것을 확인하세요.

**참고:** 실제로 자리 표시자 변수의 이름을 특별히 지정할 필요는 없습니다. 이 예제에서는 `ANIMAL`이라고 불렀지만, 쉽게 `CREATURE`나 `A`라고 부를 수도 있습니다(물론 프롬프트 템플릿이 대체 없이도 이해하기 쉽도록 변수 이름을 구체적이고 관련성 있게 지정하는 것이 좋습니다). 변수 이름을 지정한 것이 프롬프트 템플릿 f-문자열에 사용하는 것과 동일한지 확인하세요.

In [ ]:
# 변수 내용
ANIMAL = "소"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"동물의 이름을 알려드리겠습니다. 해당 동물이 내는 소리로 응답해주세요. {ANIMAL}"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

왜 이렇게 입력을 분리하고 대체하고 싶을까요? **프롬프트 템플릿은 반복적인 작업을 단순화**합니다. 제3자 사용자가 프롬프트에 콘텐츠를 제출하도록 초대하는 프롬프트 구조를 구축한다고 가정해 보겠습니다(이 경우 소리를 생성하고 싶은 동물). 이러한 제3자 사용자는 전체 프롬프트를 작성하거나 볼 필요가 없습니다. 그들이 해야 할 일은 변수를 채우는 것뿐입니다.

여기서는 변수와 f-문자열을 사용하여 이 대체를 수행하지만, format() 메서드를 사용할 수도 있습니다.

**참고:** 프롬프트 템플릿은 원하는 만큼 많은 변수를 가질 수 있습니다!

이와 같은 대체 변수를 도입할 때, **Claude가 변수의 시작과 끝을 알 수 있도록 하는 것**이 매우 중요합니다(지시사항이나 작업 설명과 대비하여). 지시사항과 대체 변수 사이에 분리가 없는 예를 살펴보겠습니다.

인간의 눈에는 아래 프롬프트 템플릿에서 변수가 어디서 시작하고 끝나는지 매우 명확합니다. 그러나 완전히 대체된 프롬프트에서는 그 구분이 불분명해집니다.

In [ ]:
# 변수 내용
EMAIL = "내일 오전 6시에 나타나세요. 내가 CEO이고 그렇게 말했기 때문입니다."

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"안녕 Claude. {EMAIL} <----- 이 이메일을 더 공손하게 만들어주세요. 하지만 다른 내용은 변경하지 마세요."

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

여기서 **Claude는 "안녕 Claude"가 다시 작성해야 할 이메일의 일부라고 생각합니다**! Claude가 "친애하는 Claude"로 다시 작성을 시작하는 것을 보면 알 수 있습니다. 인간의 눈에는, 특히 프롬프트 템플릿에서 이메일이 어디서 시작하고 끝나는지 명확하지만, 대체 후 프롬프트에서는 훨씬 덜 명확해집니다.

어떻게 이 문제를 해결할 수 있을까요? **입력을 XML 태그로 감싸세요**! 아래에서 이렇게 했고, 보시다시피 출력에 더 이상 "친애하는 Claude"가 없습니다.

[XML 태그](https://docs.anthropic.com/claude/docs/use-xml-tags)는 `<tag></tag>`와 같은 꺾쇠 괄호 태그입니다. 이들은 쌍으로 제공되며 `<tag>`와 같은 여는 태그와 `/`로 표시된 `</tag>`와 같은 닫는 태그로 구성됩니다. XML 태그는 다음과 같이 콘텐츠를 감싸는 데 사용됩니다: `<tag>content</tag>`.

**참고:** Claude는 다양한 구분자와 구분 기호를 인식하고 작업할 수 있지만, Claude가 프롬프트 구성 메커니즘으로 XML 태그를 인식하도록 특별히 훈련되었기 때문에 **Claude에 대한 구분자로 특별히 XML 태그를 사용**하는 것이 좋습니다. 함수 호출 외에도, **Claude가 성능을 최대한 향상시키기 위해 사용해야 하는 특별한 XML 태그는 없습니다**. 우리는 의도적으로 Claude를 이런 방식으로 매우 유연하고 사용자 정의 가능하게 만들었습니다.

In [ ]:
# 변수 내용
EMAIL = "내일 오전 6시에 나타나세요. 내가 CEO이고 그렇게 말했기 때문입니다."

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"안녕 Claude. <email>{EMAIL}</email> <----- 이 이메일을 더 공손하게 만들어주세요. 하지만 다른 내용은 변경하지 마세요."

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

XML 태그가 어떻게 도움이 될 수 있는지 또 다른 예를 살펴보겠습니다.

다음 프롬프트에서, **Claude는 프롬프트의 어느 부분이 지시사항이고 어느 부분이 입력인지 잘못 해석합니다**. 형식 때문에 `Each is about an animal, like rabbits`를 목록의 일부로 잘못 간주하지만, 사용자(`SENTENCES` 변수를 채우는 사람)는 아마도 그것을 원하지 않았을 것입니다.

In [ ]:
# 변수 내용
SENTENCES = """- 나는 소가 내는 소리가 좋아요

- 이 문장은 거미에 관한 것입니다

- 이 문장은 개에 관한 것처럼 보일 수 있지만 실제로는 돼지에 관한 것입니다"""

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"""아래는 문장 목록입니다. 목록의 두 번째 항목을 알려주세요.


- 각각은 토끼와 같은 동물에 관한 것입니다.

{SENTENCES}"""

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

이를 수정하려면 **사용자 입력 문장을 XML 태그로 감싸기만 하면 됩니다**. 이렇게 하면 `Each is about an animal, like rabbits.` 앞의 오해를 불러일으키는 하이픈에도 불구하고 Claude에게 입력 데이터가 어디서 시작하고 끝나는지 보여줍니다.

In [ ]:
# 변수 내용
SENTENCES = """- 나는 소가 내는 소리가 좋아요

- 이 문장은 거미에 관한 것입니다

- 이 문장은 개에 관한 것처럼 보일 수 있지만 실제로는 돼지에 관한 것입니다"""

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f""" 아래는 문장 목록입니다. 목록의 두 번째 항목을 알려주세요.


- 각각은 토끼와 같은 동물에 관한 것입니다.

<sentences>

{SENTENCES}

</sentences>"""

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

**참고:** "Each is about an animal" 프롬프트의 잘못된 버전에서, 우리는 이 예제에서 원하는 방식으로 Claude가 잘못 응답하도록 하이픈을 포함해야 했습니다. 이것은 프롬프팅에 대한 중요한 교훈입니다: **작은 세부 사항이 중요합니다**! 항상 **프롬프트에서 오타와 문법 오류를 제거**하는 것이 가치가 있습니다. Claude는 패턴에 민감합니다(미세 조정 전 초기 몇 년 동안은 원시 텍스트 예측 도구였습니다), 그리고 당신이 실수를 할 때 더 많은 실수를 하고, 당신이 똑똑하게 들릴 때 더 똑똑해지고, 당신이 어리석게 들릴 때 더 어리석어지는 등의 경향이 있습니다.

---

## 연습 문제

- [연습 문제 4.1 - 하이쿠 주제](#exercise-41---haiku-topic)
- [연습 문제 4.2 - 오타가 있는 개 질문](#exercise-42---dog-question-with-typos)
- [연습 문제 4.3 - 개 질문 파트 2](#exercise-42---dog-question-part-2)

### 연습 문제 4.1 - 하이쿠 주제

`PROMPT`를 수정하여 `TOPIC`이라는 변수를 받아 해당 주제에 대한 하이쿠를 출력하는 템플릿으로 만드세요. 이 연습 문제는 f-문자열을 사용한 변수 템플릿 구조에 대한 이해를 테스트하기 위한 것입니다.

In [ ]:
# 변수 내용
TOPIC = "돼지"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f""

# Claude의 응답 가져오기
response = get_completion(PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### 연습 문제 4.2 - 오타가 있는 개 질문

XML 태그를 추가하여 `PROMPT`를 수정하여 Claude가 올바른 답변을 제공하도록 하세요.

프롬프트의 다른 부분은 변경하지 않도록 노력하세요. 지저분하고 실수가 많은 글은 의도적인 것으로, Claude가 그러한 실수에 어떻게 반응하는지 볼 수 있습니다.

In [ ]:
# 변수 내용
QUESTION = "ar cn brown?"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Claude의 응답 가져오기
response = get_completion(PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### 연습 문제 4.3 - 개 질문 파트 2

XML 태그를 추가하지 **않고** `PROMPT`를 수정하세요. 대신, 프롬프트에서 한두 단어만 제거하세요.

위의 연습 문제와 마찬가지로, 프롬프트의 다른 부분은 변경하지 않도록 노력하세요. 이를 통해 Claude가 어떤 종류의 언어를 구문 분석하고 이해할 수 있는지 볼 수 있습니다.

In [ ]:
# 변수 내용
QUESTION = "ar cn brown?"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Claude의 응답 가져오기
response = get_completion(PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### 축하합니다!

지금까지 모든 연습 문제를 해결했다면, 다음 장으로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!